In [1]:
import os
from tqdm import tqdm

import config
import modules.dataloaders as detection_data_loader
import modules.metrics as metrics

import numpy as np
import torch
import torchmetrics
from torchmetrics.detection.mean_ap import MeanAveragePrecision

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.14 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


In [2]:
import onnx
import onnxruntime

# Check Models

In [3]:
bed_fasdd_name = './onnx_models/bed_detection_fasdd__cpu.onnx'
bed_fasdd_model = onnx.load(bed_fasdd_name)
onnx.checker.check_model(bed_fasdd_model)

In [4]:
svd_name = './onnx_models/bed_detection_fasdd__svd__cpu.onnx'
svd_model = onnx.load(svd_name)
onnx.checker.check_model(svd_model)

In [5]:
pruned_after_svd_name = './onnx_models/bed_detection_fasdd__pruned_after_svd__cpu.onnx'
pruned_after_svd_model = onnx.load(pruned_after_svd_name)
onnx.checker.check_model(pruned_after_svd_model)

# Helper function to convert pytorch tensors to numpy. Useful to handle datatset output.

In [6]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# Detection Loaders

In [7]:
detection_loader = detection_data_loader.get_val_loader()
detection_dfire_loader = detection_data_loader.get_dfire_val_loader()
detection_fasdd_uav_loader = detection_data_loader.get_fasdd_uav_val_loader()
detection_fasdd_cv_loader = detection_data_loader.get_fasdd_cv_val_loader()


TEST DFire dataset
DFire Removed wrong images: 0
DFire Removed due to overlapping: 310
DFire Removed due to more than 10: 13

Test DFire dataset len: 3983

TEST FASDD UAV dataset
FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 377
FASDD Removed due to more than 10: 156

Test FASDD UAV dataset len: 3648

TEST FASDD CV dataset
FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 317
FASDD Removed due to more than 10: 44

Test FASDD CV dataset len: 15523

Concatenate Test DFire and FASDD UAV datasets
Test dataset len: 7631
Concatenate with FASDD CV dataset
Test dataset len: 23154

TEST DFire dataset
DFire Removed wrong images: 0
DFire Removed due to overlapping: 310
DFire Removed due to more than 10: 13

Test dataset len: 3983

TEST FASDD UAV dataset
FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 377
FASDD Removed due to more than 10: 156

Test FASDD UAV dataset len: 3648

TEST FASDD CV dataset
FASDD Removed wrong images: 0
FASDD Removed due to

# Evaluate Detector

In [8]:
map_metric = MeanAveragePrecision(
    box_format='xyxy',
    iou_thresholds=[config.IOU_THRESHOLD],
    class_metrics=True, # Enables separated metrics for each class
    #average='micro',
    extended_summary=False).to('cpu')

### Evaluation Function

In [9]:
def eval_detector_onnx(loader, model_name, score_thres):

    ort_session = onnxruntime.InferenceSession(model_name, providers=["CPUExecutionProvider"])

    map_metric.reset()
    
    loop = tqdm(loader, desc='Validating', leave=True)

    for batch_idx, (img, label) in enumerate(loop):

        for idx in range(config.BATCH_SIZE):
            
            ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(img[idx].unsqueeze(dim=0))}
            out = ort_session.run(None, ort_inputs)
            
            # out of onnx session is a list: [out_tensor with batch dim] -> [ (1,12,7,7) ] 
            # -> out[0] = (1,12,7,7)
            # -> out[0][0] = (12,7,7)
            #print(f'Out type: {type(out[0])} - Output shape: {out[0].shape}')
            out = torch.tensor(np.array(out[0]))
            out = out.permute(0, 2, 3, 1)
            #print(f'Out shape after permute: {out.shape}')
            #print(f'Out shape after indexing: {out[0].shape}')
            
            # Label should be [xc, yc, w, h, score=1, smoke, fire] in 7x7 square
            #print(f'Label indexed shape: {label[idx].shape}')
            
        # Mean Average Precision
            target_boxes = metrics.get_true_boxes(label[idx].detach().to('cpu'))
            pred_boxes = metrics.get_pred_boxes(
                model_out = out[0].detach().to('cpu'),
                score_threshold=score_thres)
            map_metric.update(preds = pred_boxes, target = target_boxes)
    
    meanAP = map_metric.compute()
    map_metric.reset()

    print(f'Smoke -> AP: {meanAP["map_per_class"][0].item():.4f} - AR: {meanAP["mar_100_per_class"][0].item():.4f}')
    print(f'Fire -> AP: {meanAP["map_per_class"][1].item():.4f} - AR: {meanAP["mar_100_per_class"][1].item():.4f}')
    print(f'mAP: {meanAP["map_50"].item():.4f}')
    
    return (
        {'mAP': meanAP['map_50'].item(),
         'AP': [meanAP['map_per_class'][0].item(), meanAP['map_per_class'][1].item()],
         'AR': [meanAP['mar_100_per_class'][0].item(), meanAP['mar_100_per_class'][1].item()]
        }
    )

In [10]:
print("\n________________________________ BED FASDD _______________________________")
_ = eval_detector_onnx(
    detection_loader, 
    bed_fasdd_name,
    0.2
)
print("\n________________________________ BED FASDD: DFire _______________________________")
_ = eval_detector_onnx(
    detection_dfire_loader, 
    bed_fasdd_name,
    0.2
)
print("\n________________________________ BED FASDD:FASDD UAV _______________________________")
_ = eval_detector_onnx(
    detection_fasdd_uav_loader, 
    bed_fasdd_name,
    0.2
)
print("\n________________________________ BED FASDD:FASDD CV _______________________________")
_ = eval_detector_onnx(
    detection_fasdd_cv_loader, 
    bed_fasdd_name,
    0.2
)
print("\n________________________________ SVD _______________________________")
_ = eval_detector_onnx(
    detection_loader, 
    svd_name,
    0.2
)
print("\n________________________________ Pruned after SVD _______________________________")
_ = eval_detector_onnx(
    detection_loader, 
    pruned_after_svd_name,
    0.2
)


________________________________ BED FASDD _______________________________


Validating: 100%|████████████████████████████████████████████████████████████████████████████████| 361/361 [06:16<00:00,  1.04s/it]


Smoke -> AP: 0.6540 - AR: 0.7163
Fire -> AP: 0.6268 - AR: 0.6755
mAP: 0.6404

________________________________ BED FASDD: DFire _______________________________


Validating: 100%|██████████████████████████████████████████████████████████████████████████████████| 62/62 [01:02<00:00,  1.01s/it]


Smoke -> AP: 0.4672 - AR: 0.5801
Fire -> AP: 0.3553 - AR: 0.4413
mAP: 0.4113

________________________________ BED FASDD:FASDD UAV _______________________________


...

Smoke -> AP: 0.6914 - AR: 0.7571
Fire -> AP: 0.7040 - AR: 0.7592
mAP: 0.6977

________________________________ SVD _______________________________


Validating: 100%|████████████████████████████████████████████████████████████████████████████████| 361/361 [06:48<00:00,  1.13s/it]


Smoke -> AP: 0.6276 - AR: 0.6996
Fire -> AP: 0.6097 - AR: 0.6655
mAP: 0.6187

________________________________ Pruned after SVD _______________________________


Validating: 100%|████████████████████████████████████████████████████████████████████████████████| 361/361 [06:12<00:00,  1.03s/it]


Smoke -> AP: 0.6368 - AR: 0.7042
Fire -> AP: 0.6208 - AR: 0.6724
mAP: 0.6288


# Aladdin Metrics to Get Precision and Precision-Recall Curves

In [11]:
import modules.metrics_mAP_aladdin as aladdin_metrics

In [12]:
''' ============================
    Cell to Box Mask
============================ '''
cell2box_mask = torch.zeros((config.S, config.S, 2))
for i in range(config.S):
    for j in range(config.S):
        cell2box_mask[i,j,0] = j
        cell2box_mask[i,j,1] = i  

### Aladdin mAP modified for ONNX detection with thresholds

In [13]:
def aladdin_get_bboxes(
    loader,
    detector_model_name,
    S=config.S,
    B=config.B,
    C=config.C,
    mask=cell2box_mask,
    device='cpu',
    iou_threshold=0.5,
    score_thres=0.2,
    box_format="midpoint"):
    
    '''
    
    Return:
        - all_pred_boxes
        - all_true_boxes
        Format: [train_idx, class_prediction, prob_score, x1, y1, x2, y2]
    '''
    
    detect_session = onnxruntime.InferenceSession(detector_model_name, providers=["CPUExecutionProvider"])
    
    
    all_pred_boxes = []
    all_true_boxes = []

    # Original Code
    # make sure model is in eval before get bboxes
    #model.eval()
    train_idx = 0

    loop = tqdm(loader, desc='Get Boxes', leave=True)
    for batch_idx, (imgs, labels) in enumerate(loop):
        # Original Code
        # Inference in GPU. Move tensor to CPU in outcell_2_outboxes
        # imgs = imgs.to(device)
        # labels = labels.to(device)


        for idx in range(config.BATCH_SIZE):
            
            detect_inputs = {detect_session.get_inputs()[0].name: to_numpy(imgs[idx].unsqueeze(dim=0))}
            out = detect_session.run(None, detect_inputs)
            out = torch.tensor(np.array(out[0]))
            out = out.permute(0, 2, 3, 1)         
            
            # Original Code
            # with torch.no_grad():
            #     predictions = model(imgs)

            # Original Code
            # Remove Permute from the model
            #predictions = predictions.permute(0, 2, 3, 1) # Original Code

            #batch_size = imgs.shape[0] # Original Code

            true_bboxes = aladdin_metrics.outcell_2_outboxes(
                out_cells=labels[idx].unsqueeze(dim=0), 
                S=S, B=B, C=C, 
                mask=mask, 
                device='cpu', # Changed to cpu
                is_pred=False)
            bboxes = aladdin_metrics.outcell_2_outboxes(
                out_cells=out, 
                S=S, B=B, C=C, 
                mask=mask, 
                device='cpu', # Changed to cpu
                is_pred=True)

            for idx in range(1): # Only 1 image every time, due to ONNX prediction in CPU
                nms_boxes = aladdin_metrics.nms_yv1_getBBoxes(
                    bboxes[idx],
                    iou_threshold=iou_threshold,
                    threshold=score_thres,
                    box_format=box_format, # Midpoint, to use iou_tensor inside
                )


                # Plot some examples
                #if batch_idx == 0 and idx == 0:
                #    plot_image(x[idx].permute(1,2,0).to("cpu"), nms_boxes)
                #    print(nms_boxes)

                for nms_box in nms_boxes:
                    all_pred_boxes.append([train_idx] + nms_box)

                for box in true_bboxes[idx]:
                    # many will get converted to 0 pred, as bboxes have Conf = 1 and the rest are 0
                    if box[1] > score_thres:
                        all_true_boxes.append([train_idx] + box)

                train_idx += 1

    #model.train()
    return all_pred_boxes, all_true_boxes

# Function to Print mAP metrics

In [14]:
def print_metrics(mAP_metrics):
    mAP, avg_prec, cls_prec, cls_rec = mAP_metrics
    
    mAP_str = "mAP @0.50"
    smoke = "Smoke"
    fire = "Fire"
    
    print(f'{mAP_str:<12}' + f'{mAP:.4f}')
    print('Average Precision')
    print(f'- {smoke:<10}' + f'{avg_prec[0]:.4f}')
    print(f'- {fire:<10}' + f'{avg_prec[1]:.4f}')
    print('Class Precision')
    print(f'- {smoke:<10}' + f'{cls_prec[0]:.4f}')
    print(f'- {fire:<10}' + f'{cls_prec[1]:.4f}')  
    print('Class Recall')
    print(f'- {smoke:<10}' + f'{cls_rec[0]:.4f}')
    print(f'- {fire:<10}' + f'{cls_rec[1]:.4f}')
    print('Class F1-Score')
    smoke_f1 = 2 * (cls_prec[0] * cls_rec[0]) / (cls_prec[0] + cls_rec[0])
    fire_f1 = 2 * (cls_prec[1] * cls_rec[1]) / (cls_prec[1] + cls_rec[1])
    print(f'- {smoke:<10}' + f'{smoke_f1:.4f}')
    print(f'- {fire:<10}' + f'{fire_f1:.4f}')

### Log Path

In [15]:
log_path = './mAP_&_pr_curves/classify_&_detect/'

# BED FASDD

## Score Thres = 0.2

In [16]:
pred_boxes, true_boxes = aladdin_get_bboxes(
    detection_loader,
    detector_model_name = bed_fasdd_name,
    score_thres = 0.2
)

Get Boxes: 100%|█████████████████████████████████████████████████████████████████████████████████| 361/361 [08:41<00:00,  1.44s/it]


In [17]:
score_02_metrics = aladdin_metrics.mAP(
    log_path=log_path + 'score_thres_2e-1/',
    pred_boxes=pred_boxes,
    true_boxes=true_boxes)

mAP:@.5: 100%|███████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:10<00:00,  5.47s/it]


In [18]:
print_metrics(score_02_metrics)

mAP @0.50   0.6410
Average Precision
- Smoke     0.6549
- Fire      0.6271
Class Precision
- Smoke     0.6990
- Fire      0.6970
Class Recall
- Smoke     0.7163
- Fire      0.6755
Class F1-Score
- Smoke     0.7076
- Fire      0.6861


## Score Thres = 0.001

In [19]:
pred_boxes, true_boxes = aladdin_get_bboxes(
    detection_loader,
    detector_model_name = bed_fasdd_name,
    score_thres = 0.001
)

Get Boxes: 100%|█████████████████████████████████████████████████████████████████████████████████| 361/361 [26:12<00:00,  4.35s/it]


In [20]:
score_0001_metrics = aladdin_metrics.mAP(
    log_path=log_path + 'score_thres_2e-1/',
    pred_boxes=pred_boxes,
    true_boxes=true_boxes)

mAP:@.5: 100%|██████████████████████████████████████████████████████████████████████████████████████| 2/2 [06:14<00:00, 187.20s/it]


In [21]:
print_metrics(score_0001_metrics)

mAP @0.50   0.6662
Average Precision
- Smoke     0.6920
- Fire      0.6403
Class Precision
- Smoke     0.0151
- Fire      0.0766
Class Recall
- Smoke     0.8158
- Fire      0.7020
Class F1-Score
- Smoke     0.0296
- Fire      0.1382


In [22]:
pred_boxes, true_boxes = aladdin_get_bboxes(
    detection_dfire_loader,
    detector_model_name = bed_fasdd_name,
    score_thres = 0.001
)

Get Boxes: 100%|███████████████████████████████████████████████████████████████████████████████████| 62/62 [04:13<00:00,  4.09s/it]


In [23]:
score_0001_metrics = aladdin_metrics.mAP(
    log_path=log_path + 'score_thres_2e-1/',
    pred_boxes=pred_boxes,
    true_boxes=true_boxes)

mAP:@.5: 100%|███████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:12<00:00,  6.30s/it]


In [24]:
print_metrics(score_0001_metrics)

mAP @0.50   0.4326
Average Precision
- Smoke     0.4992
- Fire      0.3659
Class Precision
- Smoke     0.0118
- Fire      0.0717
Class Recall
- Smoke     0.7092
- Fire      0.4862
Class F1-Score
- Smoke     0.0233
- Fire      0.1250


In [41]:
pred_boxes, true_boxes = aladdin_get_bboxes(
    detection_fasdd_uav_loader,
    detector_model_name = bed_fasdd_name,
    score_thres = 0.001
)

Get Boxes: 100%|███████████████████████████████████████████████████████████████████████████████████| 57/57 [03:35<00:00,  3.78s/it]


In [42]:
score_0001_metrics = aladdin_metrics.mAP(
    log_path=log_path + 'score_thres_2e-1/',
    pred_boxes=pred_boxes,
    true_boxes=true_boxes)

mAP:@.5: 100%|███████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:12<00:00,  6.14s/it]


In [43]:
print_metrics(score_0001_metrics)

mAP @0.50   0.6210
Average Precision
- Smoke     0.6930
- Fire      0.5491
Class Precision
- Smoke     0.0159
- Fire      0.0794
Class Recall
- Smoke     0.7502
- Fire      0.5657
Class F1-Score
- Smoke     0.0310
- Fire      0.1392


In [26]:
pred_boxes, true_boxes = aladdin_get_bboxes(
    detection_fasdd_cv_loader,
    detector_model_name = bed_fasdd_name,
    score_thres = 0.001
)

Get Boxes: 100%|█████████████████████████████████████████████████████████████████████████████████| 242/242 [15:25<00:00,  3.83s/it]


In [27]:
score_0001_metrics = aladdin_metrics.mAP(
    log_path=log_path + 'score_thres_2e-1/',
    pred_boxes=pred_boxes,
    true_boxes=true_boxes)

mAP:@.5: 100%|███████████████████████████████████████████████████████████████████████████████████████| 2/2 [02:58<00:00, 89.45s/it]


In [28]:
print_metrics(score_0001_metrics)

mAP @0.50   0.7270
Average Precision
- Smoke     0.7324
- Fire      0.7216
Class Precision
- Smoke     0.0157
- Fire      0.0769
Class Recall
- Smoke     0.8601
- Fire      0.7852
Class F1-Score
- Smoke     0.0309
- Fire      0.1401


# SVD 

## Score Thres = 0.2

In [29]:
pred_boxes, true_boxes = aladdin_get_bboxes(
    detection_loader,
    detector_model_name = svd_name,
    score_thres = 0.2
)

Get Boxes: 100%|█████████████████████████████████████████████████████████████████████████████████| 361/361 [08:46<00:00,  1.46s/it]


In [30]:
score_02_metrics = aladdin_metrics.mAP(
    log_path=log_path + 'score_thres_2e-1/',
    pred_boxes=pred_boxes,
    true_boxes=true_boxes)

mAP:@.5: 100%|███████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:11<00:00,  5.75s/it]


In [31]:
print_metrics(score_02_metrics)

mAP @0.50   0.6204
Average Precision
- Smoke     0.6308
- Fire      0.6099
Class Precision
- Smoke     0.6696
- Fire      0.6482
Class Recall
- Smoke     0.6997
- Fire      0.6656
Class F1-Score
- Smoke     0.6843
- Fire      0.6568


## Score Thres = 0.001

In [32]:
pred_boxes, true_boxes = aladdin_get_bboxes(
    detection_loader,
    detector_model_name = svd_name,
    score_thres = 0.001
)

Get Boxes: 100%|█████████████████████████████████████████████████████████████████████████████████| 361/361 [22:57<00:00,  3.81s/it]


In [33]:
score_0001_metrics = aladdin_metrics.mAP(
    log_path=log_path + 'score_thres_2e-1/',
    pred_boxes=pred_boxes,
    true_boxes=true_boxes)

mAP:@.5: 100%|██████████████████████████████████████████████████████████████████████████████████████| 2/2 [06:18<00:00, 189.08s/it]


In [34]:
print_metrics(score_0001_metrics)

mAP @0.50   0.6453
Average Precision
- Smoke     0.6686
- Fire      0.6220
Class Precision
- Smoke     0.0150
- Fire      0.0687
Class Recall
- Smoke     0.8072
- Fire      0.6915
Class F1-Score
- Smoke     0.0295
- Fire      0.1249


# Pruned after SVD

## Score Thres = 0.2

In [35]:
pred_boxes, true_boxes = aladdin_get_bboxes(
    detection_loader,
    detector_model_name = pruned_after_svd_name,
    score_thres = 0.2
)

Get Boxes: 100%|█████████████████████████████████████████████████████████████████████████████████| 361/361 [08:41<00:00,  1.44s/it]


In [36]:
score_02_metrics = aladdin_metrics.mAP(
    log_path=log_path + 'score_thres_2e-1/',
    pred_boxes=pred_boxes,
    true_boxes=true_boxes)

mAP:@.5: 100%|███████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:11<00:00,  5.80s/it]


In [37]:
print_metrics(score_02_metrics)

mAP @0.50   0.6286
Average Precision
- Smoke     0.6373
- Fire      0.6199
Class Precision
- Smoke     0.6865
- Fire      0.6478
Class Recall
- Smoke     0.7046
- Fire      0.6726
Class F1-Score
- Smoke     0.6954
- Fire      0.6600


## Score Thres = 0.001

In [38]:
pred_boxes, true_boxes = aladdin_get_bboxes(
    detection_loader,
    detector_model_name = pruned_after_svd_name,
    score_thres = 0.001
)

Get Boxes: 100%|█████████████████████████████████████████████████████████████████████████████████| 361/361 [18:07<00:00,  3.01s/it]


In [39]:
score_0001_metrics = aladdin_metrics.mAP(
    log_path=log_path + 'score_thres_2e-1/',
    pred_boxes=pred_boxes,
    true_boxes=true_boxes)

mAP:@.5: 100%|██████████████████████████████████████████████████████████████████████████████████████| 2/2 [05:14<00:00, 157.06s/it]


In [40]:
print_metrics(score_0001_metrics)

mAP @0.50   0.6525
Average Precision
- Smoke     0.6738
- Fire      0.6312
Class Precision
- Smoke     0.0190
- Fire      0.0689
Class Recall
- Smoke     0.8034
- Fire      0.6975
Class F1-Score
- Smoke     0.0370
- Fire      0.1254
